In [ ]:
from twelvedata import TDClient

td = TDClient(apikey="fd7758c9dfaa4fcab8e02ebb0212b7ea")

# Definindo o intervalo de datas
data = td.time_series(
    symbol="PETR4",
    interval="1day",
    start_date="2005-01-01",
    end_date="2021-12-31",
    outputsize=5000  # Define um número maior para puxar mais dados
).as_pandas()

print(data)


                open     high       low     close    volume
datetime                                                   
2021-12-30  28.55000  28.7000  28.39000  28.45000  43229100
2021-12-29  28.70000  28.9000  28.42000  28.54000  35508400
2021-12-28  29.01000  29.2800  28.70000  28.78000  30688100
2021-12-27  28.32000  28.7800  28.15000  28.75000  44227900
2021-12-23  28.33000  28.4200  28.15000  28.33000  28713600
...              ...      ...       ...       ...       ...
2005-01-07  11.64875  11.7875  11.56375  11.67625  25980800
2005-01-06  11.65250  11.6875  11.48750  11.62500  28224000
2005-01-05  11.66250  11.7725  11.47500  11.59000  30873600
2005-01-04  11.87375  11.9500  11.62500  11.63500  37667200
2005-01-03  12.18750  12.2375  11.77375  11.77500  30044800

[4224 rows x 5 columns]
